# Define Map Environment

In [ ]:
import numpy as np
import pandas as pds
import random
import copy
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from collections import deque
from keras import backend as K
import tensorflow as tf


print(K.tensorflow_backend._get_available_gpus())
#config = tf.ConfigProto( device_count = {'GPU': 7 , 'CPU': 8} ) 
#sess = tf.Session(config=config) 
#keras.backend.set_session(sess)

class map(object):
    def __init__(self, size=10, blocks_rate=0.1):
        self.size = size if size > 3 else 10
        self.blocks = int((size ** 2) * blocks_rate) 
        self.begin_list = []
        self.map_list = []
        self.e_list = []

    def gen_map(self, k):
        if k == 0: self.map_list.append(self.begin_list)
        elif k == self.size - 1: self.map_list.append(self.e_list)
        else:
            tmp_list = []
            for l in range(0,self.size):
                if l == 0: tmp_list.extend("#")
                elif l == self.size-1: tmp_list.extend("#")
                else:
                    a = random.randint(-1, 0)
                    tmp_list.extend([a])
            self.map_list.append(tmp_list)

    def insert_blocks(self, k, begin_random, end_random):
        b_y = random.randint(1, self.size-2)
        b_x = random.randint(1, self.size-2)
        if [b_y, b_x] == [1, begin_random] or [b_y, b_x] == [self.size - 2, end_random]: k = k-1
        else: self.map_list[b_y][b_x] = "#"
            
    def generate_map(self): 
        begin_random = random.randint(1, (self.size / 2) - 1)
        for i in range(0, self.size):
            if i == begin_random: self.begin_list.extend("S")
            else: self.begin_list.extend("#")
        start_point = [0, begin_random]

        end_random = random.randint((self.size / 2) + 1, self.size - 2)
        for j in range(0, self.size):
            if j == end_random: self.e_list.extend([100])
            else: self.e_list.extend("#")
        goal_point = [self.size - 1, end_random]

        for k in range(0, self.size):
            self.gen_map(k)
        
        for k in range(self.blocks):
            self.insert_blocks(k, begin_random, end_random)

        return self.map_list, start_point, goal_point
    
class Field(object):
    def __init__(self, map, start_point, goal_point):
        self.map = map
        self.start_point = start_point
        self.goal_point = goal_point
        self.movable_vec = [[1,0],[-1,0],[0,1],[0,-1]]

    def display(self, point=None):
        field_data = copy.deepcopy(self.map)
        if not point is None:
                y, x = point
                field_data[y][x] = "{}{}"
        else:
                point = ""
        for line in field_data:
                print ("\t" + "%3s " * len(line) % tuple(line))

    def get_actions(self, state):
        movables = []
        if state == self.start_point:
            y = state[0] + 1
            x = state[1]
            a = [[y, x]]
            return a
        else:
            for v in self.movable_vec:
                y = state[0] + v[0]
                x = state[1] + v[1]
                if not(0 < x < len(self.map) and
                       0 <= y <= len(self.map) - 1 and
                       map[y][x] != "#" and
                       map[y][x] != "S"):
                    continue
                movables.append([y,x])
            if len(movables) != 0:
                return movables#append a possible state to moveables array if possible to move there
            else:
                return None

    def get_val(self, state):
        y, x = state
        if state == self.start_point: return 0, False
        else:
            v = float(self.map[y][x])
            if state == self.goal_point: 
                return v, True
            else: 
                return v, False
size = 15
barrier_rate = 0.1

map_1 = map(size, barrier_rate)
map, start_point, goal_point = map_1.generate_map()
map_field = Field(map, start_point, goal_point)

map_field.display()

# Define Deep Q-Network

In [ ]:
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.9
        self.learning_rate = 1.0
        self.e_decay = 0.9999
        self.e_min = 0.01
        self.learning_rate = 0.0001
        self.model = self.build_model()
        
    def learning_rate_refresh(self):
        self.learning_rate=4*self.learning_rate
        if self.learning_rate<0.1:
            self.learning_rate=0.25
        if self.learning_rate>1.0:
            self.learning_rate=1.0
        print("learning rate refreshed")
        



    def build_model(self):
        model = Sequential()
        model.add(Dense(128, input_shape=(2,2), activation='tanh'))
        model.add(Flatten())
        model.add(Dense(128, activation='tanh'))
        model.add(Dense(128, activation='tanh'))
        model.add(Dense(1, activation='linear'))
        model.compile(loss="mse", optimizer=RMSprop(lr=self.learning_rate))
        return model

    def remember_memory(self, state, action, reward, next_state, next_movables, done):
        self.memory.append((state, action, reward, next_state, next_movables, done))

    def choose_action(self, state, movables):
        if self.learning_rate >= random.random():
            return random.choice(movables)
        else:
            return self.choose_best_action(state, movables)
        
    def choose_best_action(self, state, movables):
        best_actions = []
        max_act_value = -100
        for a in movables:
            np_action = np.array([[state, a]])
            act_value = self.model.predict(np_action)
            if act_value > max_act_value:
                best_actions = [a,]
                max_act_value = act_value
            elif act_value == max_act_value:
                best_actions.append(a)
        return random.choice(best_actions)

    def replay_experience(self, batch_size):
        batch_size = min(batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)
        X = []
        Y = []
        for i in range(batch_size):
            state, action, reward, next_state, next_movables, done = minibatch[i]
            input_action = [state, action]
            if done:
                target_f = reward
            else:
                next_rewards = []
                for i in next_movables:
                    np_next_s_a = np.array([[next_state, i]])
                    next_rewards.append(self.model.predict(np_next_s_a))
                np_n_r_max = np.amax(np.array(next_rewards))
                target_f = reward + self.gamma * np_n_r_max
            X.append(input_action)
            Y.append(target_f)
        np_X = np.array(X)
        np_Y = np.array([Y]).T
        self.model.fit(np_X, np_Y, epochs=1, verbose=0)
        if self.learning_rate > self.e_min:
            self.learning_rate *= self.e_decay

Refreshing the parameters

In [ ]:
tracer = DQN(2,2)
not_done_count=0
done_count=0
episodes = 15000
steps = 1000

# Begin training of the Network

In [ ]:


for e in range(episodes):
    state = start_point
    score = 0
    if done_count>500:
        break
    if e%1000:
        done_count=0
    for step in range(steps):
        movables = map_field.get_actions(state)
        action = tracer.choose_action(state, movables)
        reward, done = map_field.get_val(action)
        score = score + reward
        next_state = action
        next_movables = map_field.get_actions(next_state)
        tracer.remember_memory(state, action, reward, next_state, next_movables, done)
        if done:
            done_count=done_count+1
            not_done_count=0
            if e % 100 == 0:
                print("episode: {}/{}, score: {}, learning rate: {:.2} \t steps: {}"
                        .format(e, episodes, score, tracer.learning_rate, step))
            break
        if step == (steps - 1):
            done_count=0
            not_done_count=not_done_count+1
            if e % 100 == 0:
                print("episode: {}/{}, score: {}, learning rate: {:.2} \t steps: {}"
                        .format(e, episodes, score, tracer.learning_rate, step))
            if not_done_count>1000:
                tracer.learning_rate_refresh()
                not_done_count=0
            break
            
        prev_state=state
        state = next_state
    tracer.replay_experience(32)

# The optimised path output

In [ ]:
state = start_point
score = 0
steps = 0
while True:
    steps += 1
    movables = map_field.get_actions(state)
    action = dql_solver.choose_best_action(state, movables)
    print("current state: {0} -> action: {1} ".format(state, action))
    reward, done = map_field.get_val(action)
    map_field.display(state)
    score = score + reward
    state = action
    print("current step: {0} \t score: {1}\n".format(steps, score))
    if done:
        map_field.display(action)
        print("goal!")
        break

The End